###  Trabajo proyecto final IIQ2003

## Transporte de contaminantes en piscinas mineras

#### Ecuación de Richards

La ecuación de acumulación volumetrica esta dada por:

$$ \frac{\partial \theta}{\partial t} = 
\frac{\partial}{\partial z} 
\left[
K \left( \frac{\partial h}{\partial z} + 1 \right)
\right] $$

Donde $\theta$ es el contenido volumétrico de agua, K la conductividad hidráulica, h la cabeza de presión y z la coordenada vertical.


#### Transporte mediante advección-dispersion

$$ 
\frac{\partial (\theta C)}{\partial t} =
\frac{\partial}{\partial z}
\left(
\theta D \frac{\partial C}{\partial z}
\right)
- q \frac{\partial C}{\partial z}
$$


#### Ecuaciones de Van Genuchten 

$$
\theta(h) = 
\begin{cases}
\theta_r + 
\dfrac{\theta_s - \theta_r}{\left[1 + |\alpha h|^n \right]^m}, & h < 0 \\
\theta_s, & h \geq 0
\end{cases}
$$

Obtenemos derivando el término $C(h)$
$$
\frac{\partial \theta}{\partial h} =
\frac{(- \alpha h)^{n}(1+(- \alpha h )^n)^{-1-m} \,m \cdot n \cdot(\theta_r-\theta_s) }{h}
$$

#### Conductividad Hidraulica

$$
K(h) = 
\begin{cases}
K_s S_e^l 
\left[1 - \left(1 - S_e^{1/m}\right)^m
\right]^2, & h < 0 \\
K_s, & h \geq 0
\end{cases}
$$


### Parte 1) 
### Discretización de las ecuaciones


#### Ecuación de Richards


Forma explícita de Richards en el cabezal de presión

$$
C(h)\frac{\partial h}{\partial t} = 
\frac{\partial}{\partial z} 
\left[
K(h) \left( \frac{\partial h}{\partial z} + 1 \right)
\right]
$$

Luego discretizando cada término

$$
C_{i}^j = C(h_i^j)=\frac{d\theta (h_{i}^j)}{dh}
$$

$$
C_{i}^j \frac{h_{i}^{j+1} - h_{i}^{j}}{\Delta t}
= - \left(\frac{q_{i +\frac{1}{2}}^j - q_{i -\frac{1}{2}}^j }{\Delta z} \right)
$$

$$
K_{i + \frac{1}{2}}^j =\frac{K_{i+1}^j + K_{i}^j}{2}, \qquad K_{i - \frac{1}{2}}^j =\frac{K_{i}^j + K_{i-1}^j}{2}
$$

$$
q_{i +\frac{1}{2}}^j = -K_{i + \frac{1}{2}}^j \left( \frac{h_{i+1}^j -h_{i}^j}{\Delta z} + 1 \right), \qquad q_{i -\frac{1}{2}}^j = -K_{i - \frac{1}{2}}^j \left( \frac{h_{i}^j -h_{i-1}^j}{\Delta z}+1 \right)
$$

### Formulación de la matriz tridiagonal 

$$
h_{i}^{j+1}
= h_i^j - \frac{\Delta t}{C_i^j\Delta z}\left(  q_{i +\frac{1}{2}}^j - q_{i -\frac{1}{2}}^j  \right)
$$

$$
h_{i}^{j+1}
= h_i^j  - \frac{\Delta t}{C_i^j\Delta z}\left[ K_{i + \frac{1}{2}}^j \left( \frac{h_{i+1}^j -h_{i}^j}{\Delta z} +1\right) -  K_{i - \frac{1}{2}}^j \left( \frac{h_{i}^j -h_{i-1}^j}{\Delta z} +1 \right) \right]
$$



### Condiciones de borde

#### Ecuación de Richards

CB1:

CB2:


CB3: Ecuación 4: Transporte de solutos


CB4: Ecuación 4: Transporte de solutos



### Discretización condiciones de borde

CB1:


CB2:


CB3:

CB4:


###  2) Implementación del método númerico


#### 2.1 importar módulos

In [7]:
import numpy as np
import matplotlib.pyplot as plt
# Mapas de colores
from matplotlib import cm

#### 2.2 Declaración de parámetros físicos

In [8]:
# propiedades fisicas del problema

# Parametros usados en la simulación par flujo de agua y transporte de soluto (NECESARIOS PARA RICHARDS)
#theta_s
ths = 0.38

# theta_r
thr = 0.045

# Parametro de tortuosidad
l = 0.5   #ajustado a la saturacion efectiva usando theta i

#Conductividad hidraulica saturada. 
K_s = 712.8

# Conductividad hidraulica estimado
K_est = 10

# velocidad promedio de flujo 
#q = 

n = 2.68

m = 1-1/n  

alpha = 0.1   #cm^-1

#Largo de la superficie de control

L = 70        #cm

# Coeficiente de dispersion D = D_L (dispesividad longitudinal)

D_L = 8       # cm

# Contenido volumetrico de agua inicial

th_0 = 0.10  #cm^3/cm^3

#Cabezal de presión inicial h(z,0)
h_0 = - ( (-1+ ( (-thr+ths)/(th_0 - thr) )**(1/m) )**(1/n))/(alpha)

#flujo de entrada en Z = L
q_0 = 13.69    # cm/ h 

#Borde inferior Z = 0, h(0,t)

h_fondo = h_0

#NECESARIOS PARA ECUACION DE ADVERSIÓN Y DISPERSIÓN 

#### 2.3 Definición de los parámetros de grilla y computacionales

Se le solicita resolver el problema considerando los siguientes parámetros:

* 100 nodos en la dirección radial


In [9]:
# Número de nodos
N = 100

# Crear un dominio discretizado
z=np.linspace(0, L, N)

# Espaciamiento
dz = L/(N-1)


### Definir función para calcular párametros
- Van Genuchten
- $C(h)$
- Conductividad Hidraulica

In [5]:
#Calcula la cantidad volumétrica de agua, theta_{i,j}; C(h) y la conductividad hidraulica K(h) para h<0
def cal_th_C_K(h, params):
    #Desempaquetamos array con parámetros, en esta función solo se ocupa thr,ths,alpha,m,n
    K_s, l, dt, ths, thr, alpha, K_est, dz,  m, n = params

    #theta_{i,j};Van Genuchten para h<0
    th_ij = thr + ( (ths-thr) / (( 1 + ( alpha*(-h) )**n )**m) )

    #Cálculo de dth/dh 
    C_ij = ( ((-alpha*h)**n)*(( 1 + (-alpha*h)**n )**(-1-m))*m*n*(thr-ths) ) / (h)

    #Cálculo de K(h), usa el parámetro th_ij calculado con Van Genutchen
    K_ij = K_s * ((th_ij - thr) / (ths - thr))**(l) * (1 - (1 - ((th_ij - thr) / (ths - thr))**(1/m) )**(m) )**(2)


    return th_ij, C_ij, K_ij #Para llamar a la función se debe hacer con un vector del tipo:
    #X, Y, Z = cal_th_C_K
    #X = th_ij ; Y = C_ij ; Z = K_ij 
    

#### 2.4 Definir función para rellenar matriz A - nodos interiores 
Para Poder usar FTCS debemos utilizar un espacio temporal bastante chico para suavizar los términos no lineales de la discretización. Recordando la forma matricial de la ecuación de Richards
$$
h_{i}^{j+1}
= h_i^j - \frac{\Delta t}{C_i^j\Delta z}\left(  q_{i +\frac{1}{2}}^j - q_{i -\frac{1}{2}}^j  \right)
$$

$$
h_{i}^{j+1}
= h_i^j  - \frac{\Delta t}{C_i^j\Delta z}\left[ K_{i + \frac{1}{2}}^j \left( \frac{h_{i+1}^j -h_{i}^j}{\Delta z} +1\right) -  K_{i - \frac{1}{2}}^j \left( \frac{h_{i}^j -h_{i-1}^j}{\Delta z} +1 \right) \right]
$$

$$
h_{i}^{j+1}
= h_i^j  + \Delta t \cdot \beta \left[ K_{i + \frac{1}{2}}^j \cdot h_{i+1}^j  + ( -K_{i + \frac{1}{2}}^j - K_{i - \frac{1}{2}}^j) \cdot h_i^j + K_{i - \frac{1}{2}}^j\cdot h_{i-1}^j
\right] + \Delta t \cdot\beta \cdot \Delta z (K_{i + \frac{1}{2}}^j - K_{i - \frac{1}{2}}^j) 
$$

Con $\beta = -\frac{1}{C_i^j(\Delta z)^2}$


### Condicion inicial de Haverkamp 1977

$$
\theta(0,z) = \theta_0 = 0.10 \text{cm}^3/\text{cm}^3
$$

Invirtiendo la ecuación de Van Genuchten se obtiene el valor de $h(\theta_0)$

$$
h(0,z) = h_0 
$$

### Condiciones de borde de Haverkamp 1977
 - Borde superior z = L: Flujo impuesto

$$
q(t) = q_0 = 13.69 \text{cm/h} \implies q_{\frac{1}{2}} = - q_0
$$

- Borde inferior z = 0, el cabezal de presion se mantiene cte en la superficie del relave 

$$
h(L, t) = h_{borde} = h_0
$$

In [ ]:
#La matriz cambia en cada paso temporal, por ello, definimos la funcion para construir la matriz en cada iteracion. 

def build_A_b(h, params, q0):
    K_s, l, dt, ths, thr, alpha, K_est, dz, m, n = params
    N = len(h)

    # Desempaquetamos los parametros theta, C y K para el nodo actual
    th, C, K = cal_th_C_K(h, params)

    # 2) K en caras (N-1 caras: entre i e i+1)
    K_face = 0.5 * (K[:-1] + K[1:])    # K_face[k] ~ K_{k+1/2}

    # 3) Matriz A y vector b
    A = np.zeros((N, N))
    b = np.zeros(N)

    #Nodo superior i=0: flujo impuesto q0
    K_half = K_face[0]      # K_{1/2}
    C0     = C[0]
    beta0  = 1.0 / (C0 * dz**2)

    A[0, 0] = - beta0 * K_half
    A[0, 1] =   beta0 * K_half
    b[0]    = (K_half - q0) / (C0 * dz)

    # --- Nodos interiores i = 1..N-2 ---
    for i in range(1, N-1):
        Ci   = C[i]
        beta = 1.0 / (Ci * dz**2)

        Kim  = K_face[i-1]  # K_{i-1/2}
        Kip  = K_face[i]    # K_{i+1/2}

        A[i, i-1] =  beta * Kim
        A[i, i]   = -beta * (Kim + Kip)
        A[i, i+1] =  beta * Kip

        b[i]      = (Kip - Kim) / (Ci * dz)

    # --- Nodo inferior i = N-1: Dirichlet en h ---
    # A[-1,:] = 0, b[-1] = 0 (h_new[-1] se impondrá fuera)

    return A, b

#### 2.5 - Definir función y parametros FTCS



In [6]:
def FTCS(t, h_old, params, q0, h_fondo):
    """
    t      = (t_inicial, t_max, write_interval)
    h_old  = perfil inicial h(z,0)
    params = (K_s, l, b, dt, ths, thr, alpha, K_est, dz, m, n)
    q0     = flujo impuesto en la superficie (cm/h o m/s según unidades)
    h_fondo    = valor de h en z=L (Dirichlet, igual a h0)
    """
    t1, t2, wi = t
    K_s, l, dt, ths, thr, alpha, K_est, dz, m, n = params

    t_vec  = []
    h_hist = []

    h = h_old.copy()

    while t1 < t2:
        # 1) construir A(h^j) y b(h^j)
        A, b = build_A_b(h, params, q0)

        # 2) paso FTCS explícito
        h_new = h + dt * (A @ h + b) # cambié np.linalgsolve

        # 3) imponer BC inferior Dirichlet: h(L,t) = h_b
        h_new[-1] = h_fondo

        # 4) avanzar tiempo
        t1 += dt
        h = h_new.copy()

        # 5) guardar si toca
        if (t1 % wi) < dt:
            h_hist.append(h.copy())
            t_vec.append(t1)

    return t_vec, h_hist


#### 2.6 Implementar el algoritmo acoplado de FTCS

In [ ]:
#Espaciamiento temporal (ajustar para mayor estabilidad)
dt = 1e-5

# parámetros Van Genuchten y demás
params = [K_s, l, dt, ths, thr, alpha, K_est, dz, m, n]

h_init = np.ones(N) * h_0

t_inicial = 0.0
t_max     = 10.0   
write_int = 1.0    

tiempos = [t_inicial, t_max, write_int]

t_vec, h_hist = FTCS(tiempos, h_init, params, q_0, h_fondo)

#### 2.7 - Gráfica del perfil de contaminación